In [ ]:
pip install kafka-python

In [1]:
import pandas as pd
from kafka import KafkaProducer, KafkaConsumer
from time import sleep
from json import dumps
import json

In [2]:
 producer = KafkaProducer(bootstrap_servers=['localhost:9092'], 
                         value_serializer=lambda x: 
                         dumps(x).encode('utf-8'))

In [4]:
producer.send('demo_testing2',value="{'name' : 'Yas'}")

In [5]:
import requests
#url = 'https://api.currentsapi.services/v1/latest-news'
url = 'https://api.currentsapi.services/v1/latest-news?category=politics'
# API key for authentication
api_key = 'iqypaoz95R0yVAaRAixjuGb3eGoPlge5_I8mLFHeMTzl2EeK'

# Request headers
headers = {
    'Authorization': api_key
}

# Fetch news articles from the Currents API
response = requests.get(url, headers=headers)
data = response.json()

In [6]:
articles = data.get('news', [])

In [7]:
df = pd.DataFrame(articles)

In [8]:
df.head()

,id,title,description,url,author,image,language,category,published
0,22e0335c-5c6a-44fe-a152-0dbb9d909f69,Utah Republicans to select nominee for Mitt Ro...,The Utah Republican Party is set to select a n...,https://abcnews.go.com/Politics/wireStory/utah...,ABC News,https://i.abcnewsfe.com/a/7a30b76b-dd4a-49c5-a...,en,"[politics, world]",2024-04-26 06:04:54 +0000
1,6d586144-c288-4b4d-9a04-6e544ac9c01a,Charges against Trump's 2020 'fake electors' a...,An Arizona grand jury’s indictment of 18 peopl...,https://abcnews.go.com/Politics/wireStory/char...,ABC News,https://i.abcnewsfe.com/a/ef9d28a6-a435-4494-a...,en,"[politics, world]",2024-04-26 05:42:24 +0000
2,765a976b-66ca-4d58-9ec5-8d627251016c,Trump's lawyers will grill ex-tabloid publishe...,Defense lawyers in Donald Trump’s hush money c...,https://abcnews.go.com/Politics/wireStory/trum...,ABC News,https://i.abcnewsfe.com/a/a4fb10d8-0dbf-492d-8...,en,"[politics, world]",2024-04-26 05:42:24 +0000
3,830a74d2-7af2-45cf-b721-bce3bd30f7ba,Rooting for Trump to fail has made his stock s...,NEW YORK -- Rooting for Donald Trump to fail h...,https://abcnews.go.com/Politics/wireStory/root...,Abc News,https://i.abcnewsfe.com/a/bb4fb676-a4e1-47c5-8...,en,"[politics, world]",2024-04-26 05:34:30 +0000
4,516d64c1-3d1a-41dc-8630-807d0d2e8c85,Trading Trump: Truth Social's first month of t...,With Donald Trump facing dozens of federal cri...,https://abcnews.go.com/Politics/wireStory/trad...,ABC News,https://i.abcnewsfe.com/a/92b650fd-3e11-4310-b...,en,"[politics, world]",2024-04-26 05:34:26 +0000


In [9]:
df.sample(10).to_dict(orient="records")

[{'id': '0f499e05-4054-4a81-92a6-781d8aa87ae3',
  'title': "The Latest | Hearing on claims of gag order violations in Trump's hush money trial rescheduled",
  'description': 'A change in the court schedule means Donald Trump won’t be forced off the campaign trail next week to attend a hearing in his hush money criminal trial',
  'url': 'https://abcnews.go.com/Politics/wireStory/latest-trumps-hush-money-trial-set-resume-3rd-109615264',
  'author': 'ABC News',
  'image': 'https://i.abcnewsfe.com/a/86c59738-a2ca-46ea-bb68-65c4b058a42c/wirestory_73ec7fdc224c4cb2f2507cbd5a3d4094_16x9.jpg?w=1600',
  'language': 'en',
  'category': ['politics', 'world'],
  'published': '2024-04-26 01:10:34 +0000'},
 {'id': '9ad53326-d2e0-4d8b-98cb-5a219dc96a62',
  'title': "Cruz Condemns Biden's, AOC's Responses to College Protests",
  'description': 'Sen. Ted Cruz, R-Texas, criticized the responses of President Joe Biden and Rep. Alexandria Ocasio-Cortez, D-N.Y., to protests that have occurred on numerous co

In [12]:
while True:
    dict_news=df.sample(1).to_dict(orient="records")[0]
    producer.send('demo_testing2',value=dict_news)
    sleep(1)

KeyboardInterrupt: 

In [ ]:
producer.flush()

In [ ]:
df.to_csv('articles.csv', index=False)

In [8]:


import requests
from datetime import datetime, timedelta

def get_news():
    # Define the API URL
    url = 'https://api.currentsapi.services/v1/search?category=Politics&start_date=2024-04-20T00%3A00%3A00%2B00%3A00&end_date=2024-04-25T00%3A00%3A00%2B00%3A00'

    # Calculate the date 5 days ago
    five_days_ago = datetime.now() - timedelta(days=5)
    five_days_ago_str = five_days_ago.strftime('%Y-%m-%dT00:00:00+00:00')
    today_date = datetime.now().strftime('%Y-%m-%dT00:00:00+00:00')

    # Define the parameters for the API query
    params = {
        'category': 'politics',
        'language': 'en',
        'start_date': five_days_ago_str,
        'end_date': today_date,
        'apiKey': 'iqypaoz95R0yVAaRAixjuGb3eGoPlge5_I8mLFHeMTzl2EeK'  # Replace 'YOUR_API_KEY' with your actual API key
    }

    # Make the API request
    response = requests.get(url, params=params)

    # Check if the request was successful
    if response.status_code == 200:
        data = response.json()
        return data
    else:
        print('Error:', response.status_code)
        return None

if __name__ == "__main__":
    news_data = get_news()
    if news_data:
        print(news_data)  # or process data further as needed


{'status': 'ok', 'news': [{'id': '9d6fdb26-71f1-4520-817f-ff6f54736172', 'title': 'Digital Nomads On The Rise', 'description': 'The US had 17.3 million digital nomads last year - according to MBO Partners the management buyout specialists - \xa0which is more than double the number in', 'url': 'https://www.electronicsweekly.com/blogs/mannerisms/consumer/divital-nomads-on-the-rise-2024-04/', 'author': 'David Manners', 'image': 'https://www.electronicsweekly.com/wp-content/uploads/2024/04/IMG_2944-1024x1024.jpeg', 'language': 'en', 'category': ['industry'], 'published': '2024-04-25 00:00:00 +0000'}, {'id': '5d65302c-82ba-4641-811c-f35b993a8b26', 'title': 'Formation and evolution of shale overpressure in deep Wufeng-Longmaxi Formation in southern Sichuan basin and its influence on reservoir pore characteristics', 'description': 'In the deep Longmaxi Formation shale gas reservoirs of the southern Sichuan Basin, strong overpressure is universally developed to varying degrees. However, there 

In [5]:
five_days_ago

datetime.datetime(2024, 4, 20, 0, 18, 38, 939366)

In [ ]:
datetime.now()

In [ ]:
previous_day

In [ ]:
previous_day_str

In [ ]:
datetime.now()

In [ ]:
today_date

In [ ]:
response

In [7]:
from datetime import datetime

# Convert the date string to datetime object
date_string = "April 25, 2024"
date_obj = datetime.strptime(date_string, "%B %d, %Y")

# Format the datetime object in the required format
formatted_date = date_obj.strftime("%Y-%m-%dT00:00:00+00:00")

print(formatted_date)


2024-04-25T00:00:00+00:00


Error: 401


In [13]:
import requests
import csv

def fetch_data(url, api_token):
    headers = {
        'Authorization': f'Token {api_token}',
        'Content-Type': 'application/json'
    }
    response = requests.get(url, headers=headers)
    if response.status_code == 200:
        return response.json()
    else:
        print(f"Failed to fetch data. Status code: {response.status_code}")
        return None

def save_as_csv(data, filename):
    with open(filename, 'w', newline='') as csvfile:
        fieldnames = data[0].keys() if data else []
        writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
        
        writer.writeheader()
        for row in data:
            writer.writerow(row)

if __name__ == "__main__":
    url = "https://api.currentsapi.services/v1/search?category=Politics&start_date=2024-03-20T00%3A00%3A00%2B00%3A00&end_date=2024-04-25T00%3A00%3A00%2B00%3A00"
    api_token = "iqypaoz95R0yVAaRAixjuGb3eGoPlge5_I8mLFHeMTzl2EeK"
    filename = "data.csv"

    fetched_data = fetch_data(url, api_token)
    if fetched_data:
        save_as_csv(fetched_data, filename)
        print(f"Data saved as {filename}")
    else:
        print("No data fetched.")


Failed to fetch data. Status code: 401
No data fetched.


In [32]:
from pyspark.sql import SparkSession
from datetime import datetime, timedelta
import time
#import spark
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-10_2.12:3.5.1,org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1 pyspark-shell'


spark = SparkSession.builder \
    .appName("KafkaStreamReader") \
    .config("spark.jars.packages", "org.apache.spark:spark-sql-kafka-0-10_2.12:3.5.1")\
    .getOrCreate()

# We will keep the consumer running for 10 seconds
stop_time = datetime.now() + timedelta(seconds=10)  # 10 seconds from now

df = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "localhost:9092") \
  .option("subscribe", "demo_testing2") \
  .load()


df.selectExpr("CAST(key AS STRING)", "CAST(value AS STRING)")


# Deserialize the value from Kafka as a String for now
deserialized_df = df.selectExpr("CAST(value AS STRING)")

# Query Kafka and wait 10sec before stopping pyspark
query = deserialized_df.writeStream.outputMode("append").format("console").start()
time.sleep(10)
query.stop()

24/04/26 01:19:19 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


AnalysisException: Failed to find data source: kafka. Please deploy the application as per the deployment section of Structured Streaming + Kafka Integration Guide.